In [1]:
# default_exp model.introvae
# default_cls_lvl 3

In [2]:
# hide
%load_ext autoreload
%autoreload 2

In [3]:
# hide
from fastcore.test import *

# Intro VAE

> Structure of the Hybrid Autoencoder - GAN Network


<img src="img/introvae.png" alt="Drawing" style="width: 900px;"/>
     
https://arxiv.org/pdf/1807.06358.pdf

In [4]:
# export

import torch
from torch import nn, optim
from deeptool.architecture import Encoder, Decoder
from deeptool.abs_model import AbsModel

import torch.nn.functional as F

In [5]:
# export


class IntroVAE(AbsModel):
    """
    Modification of the IntroVAE-Paper for 3-Dimensional tasks in MR-Imaging
    based on: https://arxiv.org/abs/1807.06358
    modified from: https://github.com/woxuankai/IntroVAE-Pytorch
    """

    def __init__(self, device, args):
        """
        Setup the general architecture for the IntroVAE model, composed of:
        >Encoder, Decoder<
        """
        super(IntroVAE, self).__init__(args)
        # gpu / cpu
        self.device = device

        # Encoder
        self.encoder = Encoder(args).to(self.device)
        # Decoder
        self.decoder = Decoder(args).to(self.device)

        # add further training params here...
        self.alpha = 0  # GAN
        self.beta = args.beta  # AE
        self.gamma = args.gamma  # VAE
        self.m = args.m  # margin for stopping gae learning if too far apart

        # without mean -> squarred error
        self.mse_loss = nn.MSELoss(reduction="sum")
        self.bce_loss = nn.BCELoss(reduction="sum")

        # optimizers
        self.optimizerEnc = optim.Adam(self.encoder.parameters(), lr=args.lr)
        self.optimizerDec = optim.Adam(self.decoder.parameters(), lr=args.lr)

    def reparametrisation(self, mu, log_sig2):
        """Apply the reparametrisation trick for VAE."""

        eps = torch.rand_like(mu, device=self.device)  # uniform distributed matrix
        # mean + sigma * eps
        z_latent = mu + torch.exp(torch.mul(0.5, log_sig2)) * eps
        return z_latent

    def kl_loss(self, mu, log_sig2):
        """
        KL-Divergence between two univariate gaussian distributions
        special case: compared to uniform distribution: mu2 = 0, sig2= 1
        """
        return -0.5 * torch.sum(1 - torch.pow(mu, 2) - torch.exp(log_sig2) + log_sig2)

    def ae_loss(self, x_hat, x):
        """
        sqrt(sum_i sum_j (x_ij - x_hat_ij)^2)
        pixelwise mean squared error! (sum requires to sum over one picture and the mean!)
        """
        return self.mse_loss(x_hat, x)

    def set_parameters(self, args):
        """
        Control training by setting the parameters:
        alpha, beta, gamma, m
        """
        self.alpha = args.alpha
        self.beta = args.beta
        self.gamma = args.gamma
        self.m = args.m
    
    def forward(self, data, update=True):
        """
        Get the different relevant outputs for Intro VAE training
        update=True to allow updating, update=False to keep networs constant
        return x_re (reconstructed)
        """
        # get the data
        x = self.prep(data).to(self.device)
        
        #=========== Update E ================
        self.optimizerEnc.zero_grad()
        
        # real
        z_mu, z_log_sig2 = self.encoder(x)
        z_enc = self.reparametrisation(z_mu, z_log_sig2)
        x_re = self.decoder(z_enc)
        
        # fake
        noise = torch.randn_like(z_enc, device=self.device)
        fake = self.decoder(noise)
        
        # encode again
        z_mu_re, z_log_sig2_re = self.encoder(x_re.detach())
        z_mu_fake, z_log_sig2_fake = self.encoder(fake.detach())
        
        # get losses
        loss_rec = self.ae_loss(x, x_re)
        loss_e_real_kl = self.kl_loss(z_mu, z_log_sig2)
        loss_e_rec_kl = self.kl_loss(z_mu_re, z_log_sig2_re)
        loss_e_fake_kl = self.kl_loss(z_mu_fake, z_log_sig2_fake)
        
        # combine losses
        loss_margin_e = loss_e_real_kl + (F.relu(self.m - loss_e_rec_kl) + F.relu(self.m - loss_e_fake_kl)) * self.alpha
        loss_e = loss_rec * self.beta + loss_margin_e * self.gamma
        
        if update:
            loss_e.backward()
            self.optimizerEnc.step()
        
        #========= Update G ================== 
        self.optimizerDec.zero_grad()
        
        # real
        z_mu, z_log_sig2 = self.encoder(x)
        z_enc = self.reparametrisation(z_mu, z_log_sig2)
        x_re = self.decoder(z_enc)
        
        # fake
        noise = torch.randn_like(z_enc, device=self.device)
        fake = self.decoder(noise)
        
        # encode again
        z_mu_re, z_log_sig2_re = self.encoder(x_re)
        z_mu_fake, z_log_sig2_fake = self.encoder(fake)
        
        # get losses
        loss_rec = self.ae_loss(x, x_re)
        loss_g_real_kl = self.kl_loss(z_mu, z_log_sig2)
        loss_g_rec_kl = self.kl_loss(z_mu_re, z_log_sig2_re)
        loss_g_fake_kl = self.kl_loss(z_mu_fake, z_log_sig2_fake)
        
        # combine losses
        loss_margin_g = loss_g_real_kl * (loss_g_rec_kl + loss_g_fake_kl) * self.alpha
        loss_g = loss_rec * self.beta + loss_margin_g * self.gamma
        
        if update:
            loss_g.backward()
            self.optimizerDec.step()
        
        else:
            # setup dictionary for Tracking
            tr_data = {}
            tr_data["loss_rec"] = loss_rec.item()
            tr_data["loss_e_real_kl"] = loss_e_real_kl.item()
            tr_data["loss_margin_e"] = loss_margin_e.item()
            tr_data["loss_margin_g"] = loss_margin_g.item()
            tr_data["loss_e"] = loss_e.item()
            tr_data["loss_g"] = loss_g.item()

            # Return output and tracking data
            return x_re, tr_data

In [6]:
# hide
from deeptool.train_loop import test_one_batch
from deeptool.parameters import get_all_args, compat_args

args = get_all_args()
args.pic_size = 32

In [7]:
# 3 dim test
args.model_type = "introvae"
args.dim = 3
args = compat_args(args)
test_one_batch(args)

Model-Type: introvae


In [8]:
# 2 dim test
args.dim = 2
args = compat_args(args)
test_one_batch(args)

Model-Type: introvae


In [9]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 05_abstract_model.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_bigan.ipynb.
Converted 24_mocoae.ipynb.
Converted 33_rnn_vae.ipynb.
Converted 99_index.ipynb.
